# Processing of SCIP features

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../common_.py
import sys
sys.path.append("..")
from common import *

In [71]:
import pyarrow

## Load metadata

In [54]:
data_dir = Path(os.environ["VSC_DATA_VO_USER"]) / "datasets/BBBC021"

image = pandas.read_csv(data_dir / "BBBC021_v1_image.csv")

In [58]:
image = image.drop(columns=["Image_PathName_DAPI", "Image_PathName_Tubulin", "Image_PathName_Actin", "Image_FileName_Tubulin", "Image_FileName_Actin"])

In [63]:
image.columns = ["meta_" + col.lower() for col in image.columns]

## Load data

In [73]:
data_dir = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/BBBC021/results/scip/270520220947_Week1/")

In [4]:
df = pandas.concat([
    pq.read_table(p).to_pandas()
    for p in (data_dir).glob("*.parquet")
], axis=0)

In [59]:
df["meta_filename"] = df.meta_path.apply(os.path.basename)

In [65]:
df = df.merge(image, left_on="meta_filename", right_on="meta_image_filename_dapi")

In [69]:
df = df.reset_index(drop=True)

In [74]:
pq.write_table(
    pyarrow.Table.from_pandas(df), data_dir / "features.parquet")